# SAMWISE Moment Of Inertia Estimation

In [38]:
import numpy as np

In [39]:
class RectangularPrism:
    def __init__(self, 
                 com: np.array,
                 size: np.array,
                 mass: float,
                 R=np.eye(3)) -> None:
        self.com = com   # com in CAD frame [m]
        self.R = R       # rotation matrix mapping local frame to CAD frame (columns are local axes in CAD frame)
        self.size = size # (x, y, z) size in local frame [m]
        self.mass = mass # mass [kg]
        
        assert np.abs(np.linalg.det(R) - 1) < 1e-10 # Rotation matrix should have determinant 1

    @property
    def I_com(self) -> np.array:
        # Moment of inertia about center of mass in local frame [kg m^2]
        # https://dynref.engr.illinois.edu/rem.html
        dx = self.size[0]
        dy = self.size[1]
        dz = self.size[2]

        Ixx = self.mass * (dy**2 + dz**2) / 12
        Iyy = self.mass * (dx**2 + dz**2) / 12
        Izz = self.mass * (dx**2 + dy**2) / 12

        return np.array([
            [Ixx, 0, 0],
            [0, Iyy, 0],
            [0, 0, Izz]
        ])
    
    @property
    def I_com_cad_frame(self) -> np.array:
        # Moment of inertia about the center of mass, in the cad frame [kg m^2]
        # TODO Be 100% sure this works
        return self.R @ self.I_com @ self.R.T

    def I_about_point(self, center: np.array) -> np.array:
        # Return the moment of inertia in the cad frame about a given pivot [kg m^2]
        x, y, z = self.com - center
        I_displacement = self.mass * np.array([
            [y**2 + z**2, -x * y, -x * z],
            [-x * y, x**2 + z**2, -y * z],
            [-x * z, -y * z, x**2 + y**2],
        ])
        return self.I_com_cad_frame + I_displacement

In [40]:
def print_inertial_properties(bodies: list[RectangularPrism]):
    print("\n\n----------INERTIAL PROPERTIES------------")
    m_overall = sum([body.mass for body in bodies])
    com_overall = sum([body.com * body.mass for body in bodies]) / m_overall
    print(f"Total mass: {m_overall} [g]")
    print(f"Com in cad frame: {com_overall} [mm]")

    I_overall = sum([body.I_about_point(com_overall) for body in bodies])

    values, vectors = np.linalg.eig(I_overall)

    print('')
    print("Principle moments of inertia about center of mass [g mm^2 * 10^-6]")
    print(f'{values / 1e6}')

    print('')
    print("Principle axes")
    print(f'{vectors}')

    print('')
    print("Moment of inertia about center of mass [g mm^2 * 10^-6]")
    print(I_overall / 1e6)

In [41]:
x = np.sqrt(2) / 2

bodies: list[RectangularPrism] = [
    # +X Panel 
    RectangularPrism(
        com=np.array([50, 0, 0]),
        size=np.array([2.5, 100, 215]),
        mass=300, # TODO
    ),

    # -X Panel 
    RectangularPrism(
        com=np.array([-50, 0, 0]),
        size=np.array([2.5, 100, 215]),
        mass=300, # TODO
    ),
    
    # +Y Panel 
    RectangularPrism(
        com=np.array([0, 50, 0]),
        size=np.array([100, 2.5, 215]),
        mass=150,
    ),

    # -Y Panel 
    RectangularPrism(
        com=np.array([0, -50, 0]),
        size=np.array([100, 2.5, 215]),
        mass=150,
    ),
    # +X Solar Panel
    RectangularPrism(
        com=np.array([50+85*x, -50-85*x, 0]),
        size=np.array([170, 5, 215]),
        mass=500, # TODO
        R = np.array([
            [x, x, 0],
            [-x, x, 0],
            [0, 0, 1]
        ]),
    ),

    # -X Solar Panel
    RectangularPrism(
        com=np.array([-50-85*x, 50+85*x, 0]),
        size=np.array([170, 5, 215]),
        mass=500, # TODO
        R = np.array([
            [x, x, 0],
            [-x, x, 0],
            [0, 0, 1]
        ]),
    ),
    # +Z Panel 
    RectangularPrism(
        com=np.array([0, 0, 215/2]),
        size=np.array([100, 100, 5]),
        mass=100,
    ),
    # -Z Panel
    RectangularPrism(
        com=np.array([0, 0, -215/2]),
        size=np.array([100, 100, 5]),
        mass=100,
    ),


    # BOARD STACKS
    # Raspberry Pi Unit
    RectangularPrism(
        com=np.array([0, 10, -215/2 + 32.5]),
        size=np.array([90, 60, 25]),
        mass=90,
    ),
    # Batteries 
    RectangularPrism(
        com=np.array([0, 0, 215/2  -50 -75/2]),
        size=np.array([90, 90, 75]),
        mass=1000,
    ),
    # PiCubed 
    RectangularPrism(
        com=np.array([0, 0, 215/2  - 45]),
        size=np.array([100, 100, 5]),
        mass=50,
    ),
    # ADCS Board + Reaction Wheels
    RectangularPrism(
        com=np.array([0, 0, -215/2 + 75]),
        size=np.array([100, 100, 30]),
        mass=500,
    ),
    

]

print_inertial_properties(bodies)




----------INERTIAL PROPERTIES------------
Total mass: 3740 [g]
Com in cad frame: [0.         0.24064171 0.03342246] [mm]
Principle moments of intertia about center of mass [g mm^2]
[14.61922201 41.2768466  32.35309961]
Principle axes
[[-0.71403158 -0.7000904  -0.00568663]
 [ 0.70010842 -0.71403377 -0.00199185]
 [-0.00266597 -0.0054035   0.99998185]]

Moment of intertia about center of mass [g mm^2]
[[27.68540563 13.32603264  0.        ]
 [13.32603264 28.21052846  0.06753008]
 [ 0.          0.06753008 32.35323412]]


In [26]:
R = np.array([
            [x, x, 0],
            [-x, x, 0],
            [0, 0, 1]
        ]),

np.linalg.det(R)

array([1.])